In [1]:
import pymysql
import getpass
import pandas as pd

In [2]:
pd.set_option('display.max_rows', 4)

# Get password for project database

In [3]:
password = getpass.getpass()
conn = pymysql.connect(host='mysql.guaminsects.net',user='aubreymoore',passwd=password,db='oryctes')

········


# Make trap_location.csv

In [4]:
sql = "SELECT trap_id, latitude, longitude FROM trap"
df = pd.io.sql.read_sql(sql, conn)
df.to_csv('guam_trap_location.csv', index=False)
#df.dropna(inplace=True)
df

,trap_id,latitude,longitude
0,0002MAN,13.453480,144.792470
1,1480,13.406813,144.663096
...,...,...,...
2265,X9998,13.505340,144.801650
2266,X9999,13.512230,144.807190


In [5]:
df.describe()

,latitude,longitude
count,2265.00000,2265.000000
mean,13.50297,144.794943
...,...,...
75%,13.52466,144.809890
max,13.65318,144.942150


# Make trap_catch.csv

In [6]:
# Note that the SQL contains a join which ensures that all values of trap_id exist in both the trap_location.csv and trap_catch.csv files.
sql = """
SELECT trap_visit.trap_id AS trap_id, visit_date AS end_date, male_count + female_count + unsexed_count AS ntrapped
FROM trap, trap_visit
WHERE trap_visit.trap_id = trap.trap_id
"""
df = pd.io.sql.read_sql(sql, conn)
df

,trap_id,end_date,ntrapped
0,0002MAN,2012-08-29,1
1,0002MAN,2012-09-27,0
...,...,...,...
87453,X9999,2013-07-03,0
87454,X9999,2013-07-30,0


### Date deployed needs to be added for each trap to provide a start_date for the first trapping period

In [7]:
sql = """
SELECT trap_id, date_deployed AS end_date, 0 as ntrapped
FROM trap
"""
df = df.append(pd.io.sql.read_sql(sql, conn))
#df.to_csv('guam_trap_catch.csv', index=False)
df

,trap_id,end_date,ntrapped
0,0002MAN,2012-08-29,1
1,0002MAN,2012-09-27,0
...,...,...,...
2265,X9998,2008-10-26,0
2266,X9999,2008-04-09,0


In [8]:
conn.close()

In [9]:
df.sort('end_date', ascending=False, inplace=True)
df.reset_index(drop=True,inplace=True)

#And here's a function that, for each group, appends the previous date:

def prev_dates(g):
    g.sort(columns=['end_date'])
    g['start_date'] = g.end_date.shift(-1)
    return g

#So all that's needed is to connect things:

df = df.groupby(df.trap_id).apply(prev_dates)
df

,trap_id,end_date,ntrapped,start_date
0,x0001M,2015-12-31,0,2015-08-11
1,X0002M,2015-12-31,0,2015-07-10
...,...,...,...,...
89720,x2006M,None,0,NaN
89721,x2007M,None,0,NaN


In [10]:
df = df[['trap_id', 'start_date','end_date','ntrapped']]
df.dropna(inplace=True)
df.to_csv('guam_trap_catch.csv', index=False)
df

/home/aubreymoore/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from IPython.kernel.zmq import kernelapp as app


,trap_id,start_date,end_date,ntrapped
0,x0001M,2015-08-11,2015-12-31,0
1,X0002M,2015-07-10,2015-12-31,0
...,...,...,...,...
89473,Uber Trap,2007-10-07,2007-10-08,1
89474,Uber Trap,2007-10-05,2007-10-07,2
